In [ ]:
import sys
import os

# Add parent directory to the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

# Import the module
import scripts.models as models
from scripts.models import GraphDataset
from scripts.models import GNN, GNN2, GNN3, GNN4
from torch_geometric.loader import DataLoader
import re
import torch
from scripts.models import validate
import numpy as np
from scripts.models import obtain_data_path
import matplotlib.pyplot as plt

# Visualization

Control Board

In [ ]:
# The parameters of dataset ---
data_name = 'METR-LA' # or 'PEMS-BAY'
n_nodes = 20

# GNN hyperparameters ---
GNN_class = 'GNN2'
hidden_channels = 8 # number of hidden channels
num_layers = 4 # number of hidden channels

# Chosen visualization ---
folder_name = 'archive_GNN2_MA6/include_t_True_T_past_6_T_future_12'
file_name = 'epoch_1.pt'


Code

In [ ]:
# Get parent folder path
current_folder_path = os.getcwd() # Get the current folder path
parent_folder_path = os.path.dirname(current_folder_path) # Get the parent folder path

In [ ]:
selected_model_path = parent_folder_path + f'/models/{data_name}/{folder_name}/{file_name}' 

In [ ]:
def extract_params_from_folder_name(folder_name):
    match = re.search(r'include_t_(True|False)_T_past_(\d+)_T_future_(\d+)', folder_name)
    if match:
        include_t = match.group(1) == 'True'
        T_past = int(match.group(2))
        T_future = int(match.group(3))
        return include_t, T_past, T_future
    return None, None, None

In [ ]:
# Setting up the model
include_t, T_past, T_future = extract_params_from_folder_name(folder_name)

# Setting up the model
if GNN_class == "GNN":
    if include_t is True:
        model = GNN(in_channels = T_past + 1, 
                hidden_channels = hidden_channels,
                num_layers = num_layers,
                include_t = include_t)
    else:
        model = GNN(in_channels = T_past, 
                hidden_channels = hidden_channels,
                num_layers = num_layers,
                include_t = include_t)
        
elif GNN_class == "GNN2":
    if include_t is True:
        model = GNN2(in_channels = T_past + 1, 
                hidden_channels = hidden_channels,
                num_layers = num_layers,
                include_t = include_t)
    else:
        model = GNN2(in_channels = T_past, 
                hidden_channels = hidden_channels,
                num_layers = num_layers,
                include_t = include_t)
        
elif GNN_class == "GNN3":
    if include_t is True:
        model = GNN3(in_channels = T_past + 1, 
                hidden_channels = hidden_channels,
                num_layers = num_layers,
                include_t = include_t)
    else:
        model = GNN3(in_channels = T_past, 
                hidden_channels = hidden_channels,
                num_layers = num_layers,
                include_t = include_t)
        
elif GNN_class == "GNN4":
        if include_t is True:
            model = GNN4(in_channels = T_past + 1, 
                    hidden_channels = hidden_channels,
                    num_layers = num_layers,
                    include_t = include_t)
        else:
            model = GNN4(in_channels = T_past, 
                    hidden_channels = hidden_channels,
                    num_layers = num_layers,
                    include_t = include_t)
    
# Load the parameters
model.load_state_dict(torch.load(selected_model_path))

In [ ]:
# Data loader
train_path, train_dataset, train_loader, train_loader_viz = obtain_data_path(data_name, T_past, T_future, include_t, name='train')
val_path, val_dataset, val_loader, val_loader_viz = obtain_data_path(data_name, T_past, T_future, include_t, name='val')
test_path, test_dataset, test_loader, test_loader_viz = obtain_data_path(data_name, T_past, T_future, include_t, name='test')

Evaluate the model

In [ ]:
def mae_loss(y_pred, y_true):
    return torch.mean(torch.abs(y_pred - y_true))

def mape_loss(y_pred, y_true, eps=1e-8):
    return torch.mean(torch.abs((y_true - y_pred) / (y_true + eps))) * 100

def mse_loss(y_pred, y_true):
    return torch.mean((y_pred - y_true)**2)

In [ ]:
# Evaluate the training set
train_MSE_loss = validate(model, train_loader, mse_loss)
train_MAE_loss = validate(model, train_loader, mae_loss)

print(f'Train RMSE: {np.sqrt(train_MSE_loss):.4f} [miles/hr]; Train MAE: {train_MAE_loss:.4f} [miles/hr].')

# Evaluate the validation set
val_MSE_loss = validate(model, val_loader, mse_loss)
val_MAE_loss = validate(model, val_loader, mae_loss)

print(f'Val RMSE: {np.sqrt(val_MSE_loss):.4f} [miles/hr]; Val MAE: {val_MAE_loss:.4f} [miles/hr].')

# Evaluate the test set
test_MSE_loss = validate(model, test_loader, mse_loss)
test_MAE_loss = validate(model, test_loader, mae_loss)

print(f'Test RMSE: {np.sqrt(test_MSE_loss):.4f} [miles/hr]; Test MAE: {test_MAE_loss:.4f} [miles/hr].')

## Visualize testing data:

In [ ]:
chosen_min_ahead = 30

In [ ]:
for idx, data in enumerate(test_loader_viz):
    if idx == 0:
        true_out = data.y
        pred_out = model(data)
    else:
        true_out = torch.cat((true_out, data.y), dim=0)
        pred_out = torch.cat((pred_out, model(data)), dim=0)

In [ ]:
import pandas as pd
min_ahead = ['All']
rmse_losses = [np.sqrt(test_MSE_loss)]
mae_losses = [test_MAE_loss]

for i in range(true_out.shape[1]):
    min_ahead.append((i+1)*5)
    rmse_losses.append(np.sqrt(mse_loss(pred_out[:, i], true_out[:, i]).item()))
    mae_losses.append(mae_loss(pred_out[:, i], true_out[:, i]).item())

loss_df = pd.DataFrame({
    'Min Ahead': min_ahead,
    'RMSE Loss': rmse_losses,
    'MAE Loss': mae_losses
})
loss_df

In [ ]:
idx = int((chosen_min_ahead/5) - 1)

for sensor_id in range(n_nodes):
    temp_true_out = true_out[sensor_id::n_nodes, idx]
    temp_pred_out = pred_out[sensor_id::n_nodes, idx]
    RMS_error = torch.sqrt(mse_loss(temp_pred_out, temp_true_out))
    residuals = temp_true_out - temp_pred_out

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 5))

    ax1.plot(temp_true_out.detach().numpy(), label='True Output')
    ax1.plot(temp_pred_out.detach().numpy(), label='Predicted Output')
    ax1.legend()
    ax1.set_xlabel('Time Sample Index')
    ax1.set_ylabel('Output Value [miles/hr]')
    ax1.set_title(f'True Output vs Predicted Output of Sensor#{sensor_id} on Test Set at {chosen_min_ahead} mins Ahead. \n RMS_error={RMS_error:.2f} [miles/hr]')
    ax1.grid(True)

    ax2.plot(residuals.detach().numpy(), label='Residuals', color='red')
    ax2.legend()
    ax2.set_xlabel('Time Sample Index')
    ax2.set_ylabel('Residual Value [miles/hr]')
    ax2.set_title(f'Residuals of Sensor#{sensor_id} on Test Set')
    ax2.grid(True)

    plt.show()
    